In [1]:
import sqlalchemy

print(sqlalchemy.__version__)

1.4.45


# Select

In [34]:
from sqlalchemy import create_engine, select, MetaData, Table, and_

In [3]:
engine = create_engine("sqlite:///data/census.sqlite")

In [5]:
# Criando uma conexão com o banco para executar comandos
connection = engine.connect()

In [15]:
metadata = MetaData()
census = Table('census', metadata, autoload=True, autoload_with=engine)

## Select with literal SQL

In [8]:
# Criando um statement SQL
statement = """
SELECT *
FROM census
LIMIT 10
"""

# Executando o statement e obtendo todos os resultados
results = connection.execute(statement).fetchall()
results

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547),
 ('Illinois', 'M', 3, 88868, 90037),
 ('Illinois', 'M', 4, 91947, 91111),
 ('Illinois', 'M', 5, 93894, 89802),
 ('Illinois', 'M', 6, 93676, 88931),
 ('Illinois', 'M', 7, 94818, 90940),
 ('Illinois', 'M', 8, 95035, 86943),
 ('Illinois', 'M', 9, 96436, 86055)]

## Simple select using functions

In [31]:
# Criando um SELECT sem usar os comandos literais do SQL
# No exemplo, obtendo todas as colunas da tabela census
statement = select([census])
print(statement)

SELECT census.state, census.sex, census.age, census.pop2000, census.pop2008 
FROM census


In [20]:
# Obtendo os resultado de select e limitando os resultados em 10
results = connection.execute(statement).fetchmany(size=10)
results

[('Illinois', 'M', 0, 89600, 95012),
 ('Illinois', 'M', 1, 88445, 91829),
 ('Illinois', 'M', 2, 88729, 89547),
 ('Illinois', 'M', 3, 88868, 90037),
 ('Illinois', 'M', 4, 91947, 91111),
 ('Illinois', 'M', 5, 93894, 89802),
 ('Illinois', 'M', 6, 93676, 88931),
 ('Illinois', 'M', 7, 94818, 90940),
 ('Illinois', 'M', 8, 95035, 86943),
 ('Illinois', 'M', 9, 96436, 86055)]

In [21]:
# Formas de acessar a coluna "age" da primeira linha
print(results[0]['age'])
print(results[0].age)
print(results[0][2])

0
0
0


## Select with where

In [23]:
# Executando um comando where na coluna state de census
statement = select([census])
statement = statement.where(census.columns.state == 'New York')

results = connection.execute(statement).fetchall()
results[:10]

[('New York', 'M', 0, 126237, 128088),
 ('New York', 'M', 1, 124008, 125649),
 ('New York', 'M', 2, 124725, 121615),
 ('New York', 'M', 3, 126697, 120580),
 ('New York', 'M', 4, 131357, 122482),
 ('New York', 'M', 5, 133095, 121205),
 ('New York', 'M', 6, 134203, 120089),
 ('New York', 'M', 7, 137986, 122355),
 ('New York', 'M', 8, 139455, 118653),
 ('New York', 'M', 9, 142454, 117369)]

In [32]:
states = ['New York', 'California', 'Texas']
statement = select([census])

# Obtendo as linhas de census onde state é um dos especificados na lista
# Além de .in_() existem também os métodos .between(), .like(), .startswith()
statement = statement.where(census.columns.state.in_(states))

print(statement)

SELECT census.state, census.sex, census.age, census.pop2000, census.pop2008 
FROM census 
WHERE census.state IN (__[POSTCOMPILE_state_1])


In [36]:
statement = select([census])

# Usando AND para obter registros onde state é California e sex não é M
# Além de and_ existem também as funções or_ e not_
statement = statement.where(
    and_(census.columns.state == 'California',
         census.columns.sex != 'M'
         )
)

connection.execute(statement).fetchmany(10)

[('California', 'F', 0, 239605, 274356),
 ('California', 'F', 1, 236543, 269140),
 ('California', 'F', 2, 240010, 262556),
 ('California', 'F', 3, 245739, 259061),
 ('California', 'F', 4, 254522, 255544),
 ('California', 'F', 5, 260264, 253899),
 ('California', 'F', 6, 261296, 247677),
 ('California', 'F', 7, 264083, 250117),
 ('California', 'F', 8, 270447, 233293),
 ('California', 'F', 9, 271482, 231261)]